In [25]:
import pandas as pd
from pathlib import Path
import random


def shuffle_seq(seq_list):
#用于随机排列seq
    random.shuffle(seq_list)
    return ''.join(seq_list)


#对于sequence数量过多的patent，将其按一定间隔分段输入prompt
def cut_prompts(df,total_length=1000,max_num=30,rand_num=5):
    #设置最大prompt长度和最大seq数量，防止prompt占用过多token
    prompts=[]
    seq_set=[]
    len_now=0
    num_now=0
    for index,row in df.iterrows():
        new_str='{}:{}'.format(row['new_seq_id'],row['peptide sequence'])
        #有一项不满足就新增prompt,并将一组seq随机排列多次
        if len_now+len(new_str) > total_length or num_now+1 >max_num:
            prompts.append([shuffle_seq(seq_set) for i in range(rand_num)])
            seq_set=[]
            len_now=0
            num_now=0

        len_now+=len(new_str)
        num_now+=1
        seq_set.append(new_str)
        
    prompts.append([shuffle_seq(seq_set) for i in range(rand_num)])
    return prompts

print(Path('.').absolute().parent)
print(Path('.').parent.absolute())
print((Path('.').absolute().parent.parent / 'data' / 'invention_processed.csv'))
df_seq=pd.read_csv((Path('.').absolute().parent.parent / 'data' / 'invention_processed.csv'))
patent_kind='A1'  #B
patent_num='20200277334'  #"09169290"  09273096 11684559  09265709  20220062139 10329336
patent_country='US'
df_unit=df_seq[df_seq['patent_no']==patent_country+patent_num+patent_kind]
print(len(df_unit))
prompts=cut_prompts(df_unit, max_num=1, rand_num=1)
for p in prompts:
    print(p)


/home/qcdong/codes/llamaIndex/app/tasks
/home/qcdong/codes/llamaIndex/app/tasks/patents
/home/qcdong/codes/llamaIndex/app/data/invention_processed.csv
26
['SEQ_ID_NO_18:WRRWWRRWWRWWRRWWRR']
['SEQ_ID_NO_17:RWWRWWRRWWRR']
['SEQ_ID_NO_19:RRVVRRVRRVVRRVVRVVRRVVRR']
['SEQ_ID_NO_21:VRRVVRRVVRVVRRVVRRVRRVVRRVVRVVRRVVRR']
['SEQ_ID_NO_20:RRWWRRWRRWWRRWWRWWRRWWRR']
['SEQ_ID_NO_23:RRVVRRVRRVVRRVVRVVRRVVRRVRRVVRRVVRVVRRVVRR']
['SEQ_ID_NO_22:VRRVWRRVVRVVRRWVRRVRRVWRRVVRVVRRWVRR']
['SEQ_ID_NO_25:RVVRVVRRWVRRVRRVWRRVVRVVRRWVRRVRRVWRRVVRVVRRWRVV']
['SEQ_ID_NO_24:RVVRVVRRVVRRVRRVVRRVVRVVRRVVRRVRRVVRRVVRVVRRVVRR']
['SEQ_ID_NO_26:HHHHHH']
['SEQ_ID_NO_2:IRRRRRRIRRRRRR']
['SEQ_ID_NO_3:IRRRIRRIRRRIRRIRRRIRR']
['SEQ_ID_NO_4:IRRIIRRIRRIIRRIRRIIRR']
['SEQ_ID_NO_5:VWRWVRRVWRWVRRVWRWVRR']
['SEQ_ID_NO_6:VWRWVRRVWRWVRR']
['SEQ_ID_NO_7:VVRVVRRVVRVVRR']
['SEQ_ID_NO_8:VVRVVRVVVRVVRVVVRVVRV']
['SEQ_ID_NO_9:RSRVVRSWSRV']
['SEQ_ID_NO_1:RRWVRRVRRVWRRVVRVVRRWVRR']
['SEQ_ID_NO_10:RFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRR

In [2]:
from elasticsearch import Elasticsearch

import json

es = Elasticsearch(hosts="http://192.168.1.124:29200")
INDEX = "patents"

body = {
    "query":{
        "bool":{
            "must":[
                {"term":{"publication_doc_number": patent_num}},
                {"term":{"publication_country": patent_country}}, 
                {"term":{"publication_kind":patent_kind}}               
            ]
        }
    }
}

result = es.search(index=INDEX,body=body, request_timeout=60)
#从es中获取原文
print(type(result))
res_dic=dict(result)
res_js=json.dumps(dict(result),indent=2)


<class 'elastic_transport.ObjectApiResponse'>


/tmp/ipykernel_3699560/3464205633.py:20: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  result = es.search(index=INDEX,body=body, request_timeout=60)
/tmp/ipykernel_3699560/3464205633.py:20: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index=INDEX,body=body, request_timeout=60)


In [3]:
print(res_dic.keys())
print(res_dic['hits'].keys())
print(res_dic['hits']['hits'][0].keys())
print('hits num', len(res_dic['hits']['hits']))

soucre_keys = res_dic['hits']['hits'][0]['_source'].keys()
print(f'{soucre_keys = }')
abstract = res_dic['hits']['hits'][0]['_source']['abstract']
print(f'{abstract = }')
# dict_keys(['Reference', 'Summary', 'Description', 'Others'])
description_keys = res_dic['hits']['hits'][0]['_source']['description'].keys()
print(f'{description_keys = }')

summary=res_dic['hits']['hits'][0]['_source']['description']['Summary']
print('\nsummary ', len(summary), type(summary), len(summary[0]))
print(summary[0][:500])
description = res_dic['hits']['hits'][0]['_source']['description']['Description']
print('\ndescription ', len(description), type(description))
desc_lens = []
for descr in description:
    desc_lens.append(len(descr))
print(f'{desc_lens = }')
print(description[0][:500])

print()
claim = res_dic['hits']['hits'][0]['_source']["claim"]
print(f'{len(claim) = }, {len(claim[0]) = }\n{claim[0]["claim_text"][0][:500] = }')
print(f'{claim[-1]["claim_text"][-1][-200:] =  }')
max = 2
count = 0
for c in claim:
    # print(f'{type(c["claim_text"]) = }{len(c["claim_text"]) = }')
    for c_text in c["claim_text"]:
        if 'malassezia' in c_text.lower():
            count += 1
            if count > max:
                break
            print(c_text)
print(f'{count = }')

dict_keys(['took', 'timed_out', '_shards', 'hits'])
dict_keys(['total', 'max_score', 'hits'])
dict_keys(['_index', '_type', '_id', '_score', '_ignored', '_source'])
hits num 1
soucre_keys = dict_keys(['patent_type', 'invention', 'application_country', 'application_doc_number', 'application_date', 'publication_country', 'publication_doc_number', 'publication_kind', 'publication_date', 'abstract', 'classifications', 'classifications_name', 'description', 'claim', 'peptide_sequence'])
abstract = ['Disclosed herein are novel peptides that can comprise antimicrobial, antiviral, antifungal or antitumor activity when administered to a subject.']
description_keys = dict_keys(['Reference', 'Summary', 'Description', 'Others'])

summary  1 <class 'list'> 33946
SUMMARY--Disclosed herein are peptides. Peptides disclosed herein can comprise a polypeptide sequence of Formula A, Formula B, Formula C, Formula D, Formula E, Formula F, Formula G, Formula H, Formula I, Formula J, Formula K, Formula L, For

In [4]:
max = 22
count = 0
claim_nums = ['153', '155', '156']
for c in claim:
    for c_text in c["claim_text"]:
        for c_num in claim_nums:
            if c_num in c_text.lower():
                count += 1
                if count > max:
                    break
                print(c_text)
print(f'{count = }')

153. The pharmaceutical formulation of claim 44, wherein the peptide, a metabolite thereof, or salt thereof exhibits antimicrobial activity against a Streptococcus agalactiae bacteria strain at a minimum inhibitory concentration that is at least two-fold lower than a minimum inhibitory concentration for an antimicrobial activity against a Streptococcus pneumoniae bacteria strain in vitro.
155. The pharmaceutical formulation of claim 44, wherein the peptide, a metabolite thereof, or salt thereof exhibits antimicrobial activity against a vancomycin-resistant Enterococcus faecium bacteria strain at a minimum inhibitory concentration that is at least two-fold lower than a minimum inhibitory concentration for an antimicrobial activity against a vancomycin-sensitive Enterococcus faecium bacteria strain in vitro.
156. The pharmaceutical formulation of claim 44, wherein the peptide or salt thereof comprises at least about 95% homology to the polypeptide of sequence Arg Arg Trp Val Arg Arg Val 

In [5]:
import re
from icecream import ic
ic.configureOutput(includeContext=True, argToStringFunction=str)
ic.lineWrapWidth = 120

from llama_index.core import Document

claims=['claim '+' '.join(dic['claim_text']) for dic in res_dic['hits']['hits'][0]['_source']["claim"]]
print(f'{len(claims) = }')
print(f'{claims[0][: 500] = }')
print(f'{claims[-1][-200:] = }')
claims_raw='The claim information:\n'+'\n'.join(claims)
description_raw='\nThe description information:\n'+'\n'.join(res_dic['hits']['hits'][0]['_source']["description"]["Description"])
ic(len(claims_raw), len(description_raw))
#文本预处理(方便之后的关键词匹配)
claims1=re.sub('\u2003',' ',claims_raw)
claims_real=re.sub('SEQ ID N[oO][.: ]+','SEQ_ID_NO_',claims1)
description1=re.sub('\u2003',' ',description_raw)
description_real=re.sub('SEQ ID N[oO][.: ]+','SEQ_ID_NO_',description1)
print(f'{claims_real[-200:] = }')
#claim和description存储在不同的document中
doc_claim = Document(text=claims_real)
doc_des = Document(text=description_real)

/home/qcdong/anaconda3/envs/llaIdx/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ic| 355982704.py:14 in <module>- len(claims_raw): 8748, len(description_raw): 132483


len(claims) = 28
claims[0][: 500] = 'claim 1.-43. (canceled)'
claims[-1][-200:] = 'claim 173. A method of making a kit, comprising combining the pharmaceutical formulation of claim 44 with a container.'
claims_real[-200:] = '44 in a container.\nclaim 172. The kit of claim 171, further comprising a syringe.\nclaim 173. A method of making a kit, comprising combining the pharmaceutical formulation of claim 44 with a container.'


In [6]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from llama_index.core import Settings

import torch
from llama_index.core import PromptTemplate, Settings
from llama_index.llms.huggingface.base import HuggingFaceLLM
from llama_index.llms.llama_cpp import LlamaCPP
from loguru import logger
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

BOS_TOKEN = "<s>"
EOS_TOKEN = "</s>"
INST_BEGIN = "[INST] "
INST_END = " [/INST]"

# def completion_to_prompt(completion):
#     return f"<s>[INST] {completion} [/INST]"


# def messages_to_prompt(messages):
#     prompt = BOS_TOKEN
#     for message in messages:
#         if message.role == "user":
#             prompt += f"{INST_BEGIN}{message.content}{INST_END}"
#         elif message.role == "assistant":
#             prompt += f"{message.content}{EOS_TOKEN}"

#     return prompt

SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

def load_hf_llm(
    model_path="/mnt/nas1/models/llama/pretrained_weights/llama2-7b-chat-hf",
):
    use_8bit = 1
    quantization_config_4bit = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        # bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    )
    quantization_config_8bit = BitsAndBytesConfig(
        load_in_8bit=True,
    )
    if use_8bit:
        quantization_config = quantization_config_8bit
    else:
        quantization_config = quantization_config_4bit
    logger.info(f"{use_8bit = }")
    llm = HuggingFaceLLM(
        model_name=model_path,
        tokenizer_name=model_path,
        context_window=4096,
        max_new_tokens=2048,
        # generate_kwargs={"temperature": 0.01, "do_sample": True, 'pad_token_id': 2},
        generate_kwargs={"do_sample": False},
        # generate_kwargs={"temperature": 0.01, "top_k": 50, "top_p": 0.95},
        query_wrapper_prompt=query_wrapper_prompt,
        device_map="auto",
        model_kwargs={
            "quantization_config": quantization_config,
            "torch_dtype": torch.float16,
        },
    )
    # print(llm.get_memory_footprint())
    return llm


# define embed model
os.environ["LLAMA_INDEX_CACHE_DIR"] = "/mnt/nas1/models/llama_index_cache"
Settings.embed_model = "local:/mnt/nas1/models/BAAI/bge-small-en-v1.5"
Settings.llm = load_hf_llm()

2024-04-11 09:34:51.035 | INFO     | __main__:load_hf_llm:61 - use_8bit = 1
Loading checkpoint shards: 100%|██████████| 2/2 [01:05<00:00, 32.86s/it]


In [7]:
# response = Settings.llm.complete("Hello! Can you tell me a poem about cats and dogs?")
# print(response.text.strip())

In [8]:
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)
# parse nodes
# parser = SentenceSplitter(chunk_size=256,chunk_overlap=8) #控制每个切片的长度以及相邻切片的重叠长度
# nodes = parser.get_nodes_from_documents([doc_claim,doc_des])

node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=[1024, 512, 256])
nodes = node_parser.get_nodes_from_documents([doc_claim,doc_des]) #documents

In [9]:
print(len(doc_claim.text), len(doc_des.text))
print(doc_claim.text[:100])
print()
print(doc_claim.text[-200:])
print()
print(f'{len(nodes) = }')
for node in nodes[:5]:
    print(f'{len(node.text) = }, {node.id_}')
print('...')
for node in nodes[-5:]:
   print(f'{len(node.text) = }')
print()
print(nodes[0].text[:200])


8734 132473
The claim information:
claim 1.-43. (canceled)
claim 44. A pharmaceutical formulation comprising:a. 

44 in a container.
claim 172. The kit of claim 171, further comprising a syringe.
claim 173. A method of making a kit, comprising combining the pharmaceutical formulation of claim 44 with a container.

len(nodes) = 430
len(node.text) = 2669, c4a7c168-1fdf-40d2-a48b-195200e057f0
len(node.text) = 4341, 1b753d35-037b-4813-94d4-48b5a7b12ab1
len(node.text) = 1790, dc8620c0-b1a0-4100-bda0-f044396afcb2
len(node.text) = 1138, e2d96306-48fb-46f4-b85a-068b650f377a
len(node.text) = 1530, 41337420-185f-49bd-b360-c9bcae572c77
...
len(node.text) = 795
len(node.text) = 804
len(node.text) = 904
len(node.text) = 190
len(node.text) = 285

The claim information:
claim 1.-43. (canceled)
claim 44. A pharmaceutical formulation comprising:a. a peptide or salt thereof comprising from about 70% to about 100% homology to a polypeptide of seque


In [10]:
# define storage context
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage import StorageContext

docstore = SimpleDocumentStore()
# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)

In [11]:
#层次索引需要载入底层的叶子节点
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes, get_deeper_nodes
from pandas import DataFrame

print(f'{len(nodes) = }')
leaf_nodes = get_leaf_nodes(nodes)
print(f'{len(leaf_nodes) = }')
mid_nodes = get_deeper_nodes(nodes)
print(f'{len(mid_nodes) = }')
root_nodes = get_root_nodes(nodes)
print(f'{len(root_nodes) = }')

leaf_nodes_lens = [len(node.text) for node in leaf_nodes]
df = DataFrame(leaf_nodes_lens, columns=['leaf_nodes_length'])
print(df.describe())
mid_nodes_lens = [len(node.text) for node in mid_nodes]
df = DataFrame(mid_nodes_lens, columns=['mid_nodes_length'])
print(df.describe())
## among the first 5 nodes, 3 is in the root nodes.
# for node, root in zip(nodes[:5], root_nodes[:5]):
#     print(f'{len(node.text) = }, {node.id_}, {node.text[:100]}')
#     print(f'{len(root.text) = }, {root.id_}, {root.text[:100]}')
#     print('')


len(nodes) = 430
len(leaf_nodes) = 258
len(mid_nodes) = 123
len(root_nodes) = 49
       leaf_nodes_length
count         258.000000
mean          566.968992
std           325.916557
min            46.000000
25%           310.000000
50%           572.000000
75%           795.000000
max          1313.000000
       mid_nodes_length
count        123.000000
mean        1162.544715
std          638.462436
min           47.000000
25%          674.000000
50%         1213.000000
75%         1570.500000
max         2423.000000


In [12]:
## Load index into vector index
from llama_index.core import VectorStoreIndex

# 利用叶子节点计算相似度，并关联到对应的父节点
base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
)

In [13]:
# import QueryBundle
from llama_index.core import QueryBundle

# import NodeWithScore
from llama_index.core.schema import NodeWithScore, BaseNode

# Retrievers
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
)
from typing import List

In [14]:
from nltk import word_tokenize 

class CustomRetriever(BaseRetriever):
    """自定义的混合索引类：相似度索引+关键词索引"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        vector_retriever_large: VectorIndexRetriever,
        max_num_keyword_nodes=3,
        keywords=[]
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever #头部相似度节点
        self._vector_retriever_large = vector_retriever_large #更大范围的头部相似度节点(用于关键词搜索)

        self.keywords=keywords #索引依据的关键词
        self.max_num_keyword_nodes=max_num_keyword_nodes #设置最大关键词节点数量
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        #利用两个不同的参数的retriever进行retrieve
        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        vector_nodes_large = self._vector_retriever_large.retrieve(query_bundle)

        #确保集合中的节点id对应的相似度由大到小排列
        vector_ids = {n.node_id for n in sorted(vector_nodes,key=lambda node: node.score,reverse=True)} 
        # vector_ids_large = {n.node_id for n in vector_nodes_large}

        #对于更大范围的相似度索引结果，取出其中含有关键词的节点
        keyword_ids = []
        for n in sorted(vector_nodes_large,key=lambda node: node.score,reverse=True):
            for k in self.keywords:
                if(k in word_tokenize(n.get_content())):
                    #判断关键词是否在文章片段的分词结果中
                    keyword_ids.append(n.node_id)
                    break

        combined_dict = {n.node_id: n for n in vector_nodes}
        combined_dict.update({n.node_id: n for n in vector_nodes_large if n.node_id in keyword_ids })

        #合并两组节点
        if(keyword_ids==[]):
            #不含有关键词的情况下照常进行相似度索引
            retrieve_ids = vector_ids
        else:
            keyword_ids_top=set(keyword_ids[:self.max_num_keyword_nodes]) #取相似度最高的几个关键词节点
            vector_ids_unique=vector_ids-keyword_ids_top  #top相似度集合中独有的节点
            retrieve_ids=keyword_ids_top #关键词集合和top相似度集合共有的节点+关键词集合中独有的节点
            add_num=len(vector_ids)-len(keyword_ids_top)
            retrieve_ids=set(list(vector_ids_unique)[:add_num]).union(retrieve_ids) #额外添加部分top相似度集合中独有的节点

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [15]:
for p in prompts:
    print(p)
print('')
print(prompts[15])
print(prompts[15][0].split('\n'))


['SEQ_ID_NO_18:WRRWWRRWWRWWRRWWRR']
['SEQ_ID_NO_17:RWWRWWRRWWRR']
['SEQ_ID_NO_19:RRVVRRVRRVVRRVVRVVRRVVRR']
['SEQ_ID_NO_21:VRRVVRRVVRVVRRVVRRVRRVVRRVVRVVRRVVRR']
['SEQ_ID_NO_20:RRWWRRWRRWWRRWWRWWRRWWRR']
['SEQ_ID_NO_23:RRVVRRVRRVVRRVVRVVRRVVRRVRRVVRRVVRVVRRVVRR']
['SEQ_ID_NO_22:VRRVWRRVVRVVRRWVRRVRRVWRRVVRVVRRWVRR']
['SEQ_ID_NO_25:RVVRVVRRWVRRVRRVWRRVVRVVRRWVRRVRRVWRRVVRVVRRWRVV']
['SEQ_ID_NO_24:RVVRVVRRVVRRVRRVVRRVVRVVRRVVRRVRRVVRRVVRVVRRVVRR']
['SEQ_ID_NO_26:HHHHHH']
['SEQ_ID_NO_2:IRRRRRRIRRRRRR']
['SEQ_ID_NO_3:IRRRIRRIRRRIRRIRRRIRR']
['SEQ_ID_NO_4:IRRIIRRIRRIIRRIRRIIRR']
['SEQ_ID_NO_5:VWRWVRRVWRWVRRVWRWVRR']
['SEQ_ID_NO_6:VWRWVRRVWRWVRR']
['SEQ_ID_NO_7:VVRVVRRVVRVVRR']
['SEQ_ID_NO_8:VVRVVRVVVRVVRVVVRVVRV']
['SEQ_ID_NO_9:RSRVVRSWSRV']
['SEQ_ID_NO_1:RRWVRRVRRVWRRVVRVVRRWVRR']
['SEQ_ID_NO_10:RFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVR']
['SEQ_ID_NO_12:KVVSSIIEIISSVVKVVSSIIEIISSVV']
['SEQ_ID_NO_11:RRTYSRSRRTYSRSRRTYSR']
['SEQ_ID_NO_14:VVRVVRRVVRV

In [16]:
from llama_index.core.retrievers.auto_merging_retriever import AutoMergingRetriever
base_retriever = base_index.as_retriever(similarity_top_k=6)
large_retriever = base_index.as_retriever(similarity_top_k=len(leaf_nodes)//2)

print(f'{prompts[15][0] = }')
keywords=[i.split(':')[0] for i in prompts[15][0].split('\n')]
print(f'{keywords = }')
custom_retriever = CustomRetriever(base_retriever, large_retriever,max_num_keyword_nodes=3,keywords=[i.split(':')[0] for i in prompts[15][0].split('\n')])  
#创建混合索引实例
retriever = AutoMergingRetriever(custom_retriever, storage_context, verbose=True)

prompts[15][0] = 'SEQ_ID_NO_7:VVRVVRRVVRVVRR'
keywords = ['SEQ_ID_NO_7']


In [17]:
from llama_index.core.query_engine import RetrieverQueryEngine
custom_query_engine = RetrieverQueryEngine.from_args(retriever)

In [21]:
#修改默认的prompt内容
from llama_index.core.prompts import PromptTemplate
qa_prompt_tmpl_str = (
    """You are a biologist and patent expert. 
    You will be provided with some contents from a patent and will be asked to answer specific questions related to the patent. 
    Please answer the question only using the provided contents and do not make up the answer with prior knowledge."""
    "---------------------\n"
    "Content is :"
    "{context_str}\n"
    "---------------------\n"
    "Given the provided content, please answer the query:\n"
    "Query: {query_str}\n"
    "Answer: "
)

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

custom_query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [22]:
from IPython.display import Markdown, display
question1="Is this patent describing agents (e.g. molecules) for inhibiting Malassezia fungus? Such applications should be claimed in the “Claims” of the patent. Please make sure the patent is specific about inhibition of Malassezia. Finally, plesae answer 'Yes' or 'No' first, then explain the reason in next line." #Please focuse on the given context and don't use prior knowledge.
response = custom_query_engine.query(question1)
display(Markdown(f"<b>{response}</b>"))

> Merging 2 nodes into parent node.
> Parent node id: a609603b-43eb-47cd-9611-4c72c782577e.
> Parent node text: To this solution will be added an exemplary peptide of SEQ_ID_NO_1, SEQ_ID_NO_2, SEQ_ID_NO_3, SEQ...



/home/qcdong/anaconda3/envs/llaIdx/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/qcdong/anaconda3/envs/llaIdx/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<b>Yes, the patent is describing agents (e.g. molecules) for inhibiting Malassezia fungus. The patent specifically mentions Malassezia in the "Background of the Invention" section, and the "Summary of the Invention" section highlights the need for effective treatments against Malassezia infections. The "Claims" section also includes claims related to the use of the described peptides for inhibiting Malassezia growth.
Reason: The patent explicitly mentions Malassezia in multiple sections, including the "Background of the Invention" and the "Summary of the Invention," and includes claims related to the use of the described peptides for inhibiting Malassezia growth. This indicates that the patent is indeed describing agents for inhibiting Malassezia fungus.</b>